In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d

In [2]:
# utilities:
def load_spectrum(filename):
  number_channels = 1016
  # print(f'loading data from : "{filename}"')
  f = open(filename, 'r')
  data = [[float(item2) for item2 in item.split('\t') if len(item2) > 0]
    for item in f.read().split('\n') if len(item) > 0]
  data_unique = set([len(item) for item in data])
  if len(data) != number_channels:
    print(f'ERROR: Total number of channels is not {number_channels}.')
    exit()
  elif data_unique == {2} or data_unique == {3}:
    energy = np.array([item[0] for item in data])
    counts = np.array([item[1] for item in data])
    if 'Exp' in filename:
      # i.e., if spectrum is NaI generated, normalize w.r.t aquisition time:
      aquisition_time = int([item for item in filename.split('_') if 'sec' in item][0].split('sec')[0])
      counts = counts/aquisition_time
    if data_unique == {2}:
      percentage_error = np.empty(energy.shape)*np.nan
    elif data_unique == {3}:
      percentage_error = np.array([item[1] for item in data])
    return energy, counts, percentage_error
  else:
    print('ERROR: At some point or all, spectrum data format does not match with either "Energy\tcounts" or "Energy\tcounts\tError".')
    exit()
  print('ERROR: cannot load spectrum data')
  exit()

In [3]:
# initialization:
class_labels = {
  'bkg':   0,
  'Ba133': 1,
  'Cs137': 2,
  'Co60':  3
}
num_classes = len(class_labels)

In [4]:
for combination in [['D0', 'train'], ['D1', 'test']]:
  print('\n', combination)
  directory, datafilename = combination[0], combination[1]

  csv = []  # to collect produced spectrums
  ID = -1

  rn_tag = 'bkg_Exp_base'
  filename = [item for item in os.listdir('base_spectrums') if rn_tag in item][0]
  spect0 = load_spectrum(f'base_spectrums/{filename}')

  # signal_spectrum:
  for RN in [*class_labels]:
    rn_tag = 'bkg_Exp_base' if RN == 'bkg' else f'{RN}_FLUKA_base'
    filename = [item for item in os.listdir('base_spectrums') if rn_tag in item][0]
    spect1 = load_spectrum(f'base_spectrums/{filename}')

    '''
    NOTE:
    min and max energy values are same for simulated and experimental data
    #energy_bins (i.e #channels or #features) are also same for simulated and experimental data
    however, the energy bin boundaries are not exactly same
    hence, for mixing the spectrums (simulated + experimental), this should be considered.
    (class-0 is always experimenal data since background can be simulated as its origin is natural radioactivity in environment)
    '''
    ene_pairs = []
    for i in range(len(spect0[0])):
      j = np.argmin(np.abs(spect1[0] - spect0[0][i]))
      ene_pairs.append([i,j])
    
    # normalize spect w.r.t total counts:
    spect0_norm = [spect0[0], spect0[1]/spect0[1].sum(), spect0[2]]
    spect1_norm = [spect1[0], spect1[1]/spect1[1].sum(), spect1[2]]
    
    if directory == 'D0':
      np.random.seed(0)
    elif directory == 'D1':
      np.random.seed(10)
    
    '''
    Now,
    the SNR (signal to noise mixing ratio) ~ random.uniform(low=0.05, high=1.1)
    '''
    
    SNR_list = [0.05] + list(np.random.uniform(low=0.05, high=1.1, size=30)) + [1.1]
    # the superimposed spetrum:
    # energy-wise addition:
    for SNR in SNR_list:
      counts = []
      for ene in ene_pairs:
        counts.append(spect0_norm[1][ene[0]] + SNR*spect1_norm[1][ene[1]])
      counts = np.array(counts)
      spect_norm = [spect0[0], counts/counts.sum(), spect0[2]]
      
      '''
      Now,
      the gross count variation:
      '''
      gross_counts_list = [500] + list(np.random.uniform(low=500, high=10000, size=30)) + [10000]
      for gross_counts in gross_counts_list:
        # note: at this point, counts have been scaled but the statistics has not changed.
        # to change the data_statistics, we resample each count from Poission distribution with new_count as expected_count
        good_spect = [spect_norm[0], spect_norm[1]*gross_counts, spect_norm[2]]
        counts = []
        for i in range(len(good_spect[1])):
          counts.append(np.random.poisson(lam=good_spect[1][i]))
        counts = np.array(counts)
        bad_spect_norm = [spect_norm[0], counts/counts.sum(), spect_norm[2]]
    
        ID += 1
        csv.append([ID] + [f'{item}' for item in bad_spect_norm[1]] + [class_labels[RN]] + [SNR] + [gross_counts])


  np.random.shuffle(csv)
  np.random.shuffle(csv)
  csv = csv[:1000*num_classes]

  csv = [['ID'] + [f'feature_{item}' for item in range(1016)] + ['label'] + ['SNR'] + ['gross_counts']] + csv

  f = open(f'{directory}/{datafilename}{directory}_detailed.csv', 'w')
  f.write('\n'.join([','.join([str(item2) for item2 in item]) for item in csv]))
  f.close()

  f = open(f'{directory}/{datafilename}{directory}.csv', 'w')
  f.write('\n'.join([','.join([str(item2) for item2 in item[:-2]]) for item in csv]))
  f.close()

  # print number of data generated for each class:
  f = open(f'{directory}/{datafilename}{directory}.csv', 'r')
  data = [int(item.split(',')[-1]) for item in f.read().split('\n')[1:]]
  f.close()

  for i in range(num_classes):
    print(f'#datapoints in class-{i}:', len([item for item in data if item == i]))
print('complete')


 ['D0', 'train']
#datapoints in class-0: 999
#datapoints in class-1: 1000
#datapoints in class-2: 1006
#datapoints in class-3: 995

 ['D1', 'test']
#datapoints in class-0: 1002
#datapoints in class-1: 1004
#datapoints in class-2: 999
#datapoints in class-3: 995
complete


In [5]:
# generating experimental data with (olny) SNR variation:

directory, datafilename = 'D2', 'test'

csv = []  # to collect produced spectrums
ID = -1

rn_tag = 'bkg_Exp_base'
filename = [item for item in os.listdir('base_spectrums') if rn_tag in item][0]
spect0 = load_spectrum(f'base_spectrums/{filename}')

# signal_spectrum:
for RN in [*class_labels]:
  rn_tag = f'{RN}_Exp_base'
  filename = [item for item in os.listdir('base_spectrums') if rn_tag in item][0]
  spect1 = load_spectrum(f'base_spectrums/{filename}')
  # since, the during the experimental aquisition of data,
  # 'bkg' (class-0) is inherently included in all signals (other classes)
  # hence, for the purpose to generate a fixed SNR spectrum, the original 'bkg' should be subtracted:
  if RN != 'bkg': spect1 = [spect1[0], spect1[1]-spect0[1], spect1[2]]  # bkg subtraction for Exp spectrums

  # normalize spect w.r.t total counts:
  spect0_norm = [spect0[0], spect0[1]/spect0[1].sum(), spect0[2]]
  spect1_norm = [spect1[0], spect1[1]/spect1[1].sum(), spect1[2]]

  if directory == 'D2':
    np.random.seed(20)

  '''
  Now,
  the SNR (signal to noise mixing ratio) ~ random.uniform(low=0.05, high=1.1)
  '''
  SNR_list = list(np.random.uniform(low=0.05, high=1.1, size=1000))

  # the superimposed spetrum:
  # energy-wise addition:
  for SNR in SNR_list:
    counts = spect0_norm[1] + SNR*spect1_norm[1]
    spect_norm = [spect0[0], counts/counts.sum(), spect0[2]]
    
    ID += 1
    csv.append([ID] + [f'{item}' for item in spect_norm[1]] + [class_labels[RN]] + [SNR] + [''])


np.random.shuffle(csv)
np.random.shuffle(csv)

csv = [['ID'] + [f'feature_{item}' for item in range(1016)] + ['label'] + ['SNR'] + ['gross_counts']] + csv

f = open(f'{directory}/{datafilename}{directory}_detailed.csv', 'w')
f.write('\n'.join([','.join([str(item2) for item2 in item]) for item in csv]))
f.close()

f = open(f'{directory}/{datafilename}{directory}.csv', 'w')
f.write('\n'.join([','.join([str(item2) for item2 in item[:-2]]) for item in csv]))
f.close()


In [6]:
# data generation: experimental + counting_statistics variation

directory, datafilename = 'D3', 'test'

csv = []  # to collect produced spectrums
ID = -1

for RN in [*class_labels]:
  rn_tag = f'{RN}_Exp_base'
  filename = [item for item in os.listdir('base_spectrums') if rn_tag in item][0]
  spect1 = load_spectrum(f'base_spectrums/{filename}')
  
  # normalize spect w.r.t total counts:
  spect1_norm = [spect1[0], spect1[1]/spect1[1].sum(), spect1[2]]

  if directory == 'D3':
    np.random.seed(30)

  '''
  Now,
  the gross count variation:
  '''
  gross_counts_list = list(np.random.uniform(low=500, high=10000, size=1000))
  for gross_counts in gross_counts_list:
    good_spect = [spect1_norm[0], spect1_norm[1]*gross_counts, spect1_norm[2]]
    counts = []
    for i in range(len(good_spect[1])):
      counts.append(np.random.poisson(lam=good_spect[1][i]))
    counts = np.array(counts)
    bad_spect_norm = [spect1_norm[0], counts/counts.sum(), spect1_norm[2]]
    
    ID += 1
    csv.append([ID] + [f'{item}' for item in bad_spect_norm[1]] + [class_labels[RN]] + [''] + [gross_counts])


np.random.shuffle(csv)
np.random.shuffle(csv)

csv = [['ID'] + [f'feature_{item}' for item in range(1016)] + ['label'] + ['SNR'] + ['gross_counts']] + csv

f = open(f'{directory}/{datafilename}{directory}_detailed.csv', 'w')
f.write('\n'.join([','.join([str(item2) for item2 in item]) for item in csv]))
f.close()

f = open(f'{directory}/{datafilename}{directory}.csv', 'w')
f.write('\n'.join([','.join([str(item2) for item2 in item[:-2]]) for item in csv]))
f.close()


In [7]:
'''
generating dataset: D4
'''

directory, datafilename = 'D42', 'test'

drift_list = [-5, -50]

for drift in drift_list:
  print('\ndrfit =', drift)

  csv = []  # to collect produced spectrums
  ID = -1

  if directory == 'D42':
    np.random.seed(42)

  # the SNR (signal to noise mixing ratio) ~ random.uniform(low=0.05, high=1.1)
  SNR_list = [0.05] + list(np.random.uniform(low=0.05, high=1.1, size=30)) + [1.1]

  for SNR in SNR_list:

    # the gross count variation:
    gross_counts_list = [500] + list(np.random.uniform(low=500, high=10000, size=30)) + [10000]

    for gross_counts in gross_counts_list:

      # for each class:
      for RN in [*class_labels]:

        # noise:
        rn_tag = 'bkg_Exp_base'
        filename = [item for item in os.listdir('base_spectrums') if rn_tag in item][0]
        spect0 = load_spectrum(f'base_spectrums/{filename}')
        spect0_norm = [spect0[0], spect0[1]/spect0[1].sum(), spect0[2]]

        # signal:
        rn_tag = f'{RN}_Exp_base'
        filename = [item for item in os.listdir('base_spectrums') if rn_tag in item][0]
        spect1 = load_spectrum(f'base_spectrums/{filename}')
        spect1_norm = [spect1[0], spect1[1]/spect1[1].sum(), spect1[2]]

        # SNR:
        counts = []
        counts = spect0_norm[1] + SNR*spect1_norm[1]
        spect_norm = [spect0[0], counts/counts.sum(), spect0[2]]

        # statistics:
        good_spect = [spect_norm[0], spect_norm[1]*gross_counts, spect_norm[2]]
        counts = []
        for i in range(len(good_spect[1])):
          counts.append(np.random.poisson(lam=good_spect[1][i]))
        counts = np.array(counts)
        bad_spect_norm = [spect_norm[0], counts/counts.sum(), spect_norm[2]]

        # energy-drift:
        old_energy = bad_spect_norm[0]
        new_energy = old_energy*(1+drift/100)
        interpolator = interp1d(bad_spect_norm[0], bad_spect_norm[1], kind='linear', fill_value="extrapolate")
        counts = interpolator(new_energy)
        spect_norm = [bad_spect_norm[0], counts/counts.sum(), bad_spect_norm[2]]

        ID += 1
        csv.append([ID] + [f'{item}' for item in spect_norm[1]] + [class_labels[RN]] + [SNR] + [gross_counts] + [drift])

  np.random.shuffle(csv)
  np.random.shuffle(csv)
  csv = csv[:1000*num_classes]

  csv = [['ID'] + [f'feature_{item}' for item in range(1016)] + ['label'] + ['SNR'] + ['gross_counts'] + ['drift%']] + csv

  f = open(f'{directory}/{datafilename}{directory}_{drift}percent_detailed.csv', 'w')
  f.write('\n'.join([','.join([str(item2) for item2 in item]) for item in csv]))
  f.close()

  f = open(f'{directory}/{datafilename}{directory}_{drift}percent.csv', 'w')
  f.write('\n'.join([','.join([str(item2) for item2 in item[:-3]]) for item in csv]))
  f.close()

  f = open(f'{directory}/{datafilename}{directory}_{drift}percent.csv', 'r')
  data = [int(item.split(',')[-1]) for item in f.read().split('\n')[1:]]
  f.close()

  for i in range(num_classes):
    print(f'class-{i}:', len([item for item in data if item == i]))



drfit = -5
class-0: 1001
class-1: 1000
class-2: 1001
class-3: 998

drfit = -50
class-0: 1001
class-1: 1000
class-2: 1001
class-3: 998
